# AWS DevDay Seoul 2019
## 모두를 위한 컴퓨터 비전 딥러닝 툴킷, GluonCV 따라하기
## Lab 3. Object detection 모델을 AWS DeepLens에 배포하기

## <font color='red'>Lab 3은 제공된 AWS 계정을 이용해서 그룹이 함께 수행합니다</font>. 

각 그룹마다 1개씩의 DeepLens를 사용할 수 있습니다.

랩 가이드에 있는 그룹에 할당된 계정을 사용해서 AWS 콘솔에 로그인 한 후, SageMaker notebook instance로 접근해서 Task 1부터 수행합니다.

![](./images/devday-deeplens-lab-19.png)

### 랩 순서

1. 제공된 USB에 있는 모델 파일을 각 그룹의 S3 버킷에 업로드합니다.
2. DeepLens 모델 정의하기
3. 제공된 DeepLens 프로젝트에서 모델 교체하기
4. DeepLens에 배포하고 테스트하기

DeepLens에 배포한 모델을 저장할 S3 버킷이름과 모델 파일 이름을 정합니다.

> 주의: S3 버킷 이름은 꼭 **"deeplens-"**로 시작해야합니다.

In [ ]:
S3_BUCKET = 'deeplens-devday-seoul-hol-group1' # 마지막 숫자를 각 그룹의 번호(1~8)로 바꿔주세요.
DL_MODEL_NAME = 'model-to-go.tar.gz'

# Task 1 - 제공된 USB의 모델 파일을 각 그룹의 S3 버킷에 업로드 하기

1. 주피터 노트북 왼쪽 메뉴의 '업로드' 아이콘을 클릭해서 USB에 있는 'model-to-go.tar.gz'를 노트북 인스턴스로 업로드합니다.

![](./images/devday-deeplens-lab-12.png)

2. 다음 명령을 수행해서 S3로 업로드 합니다.

In [ ]:
!aws s3 cp ./$DL_MODEL_NAME s3://$S3_BUCKET/models/

# Task 2 - DeepLens 모델 정의하기

> ## **주의:** 강연자의 별도의 안내가 있을 경우, Task 2를 수행하지 않고 Task 3을 수행합니다.

1. DeepLens 콘솔로 이동 후, 'Resources > Models' 선택
2. 'Import model' 버튼 클릭

![](./images/devday-deeplens-lab-01.png)

3. 'Import source'를 'Externally trained model'로 변경
4. Model settings에 값들을 입력
- Model artifact path: s3://__S3_BUCKET__/models/model-to-go.tar.gz <font color='red'>(맨 처음에 정의한 S3_BUCKET 이름으로 교체)</font>
- Model name: model 이름 (예: __model-deeplens-ssd__)
- Model framework: MXNet 선택

![](./images/devday-deeplens-lab-02.png)

# Task 3 - 제공된 DeepLens 프로젝트에서 모델 교체하기

1. DeepLens 콘솔로 이동 후, 'Resources > Projects' 선택하고,
2. 프로젝트 목록에서 '__Object-detection__'을 클릭합니다.

![](./images/devday-deeplens-lab-13.png)

3. 'Edit' 클릭 후, 기존 모델의 삭제를 위해서 Model의 'Remove' 버튼을 클릭합니다.

![](./images/devday-deeplens-lab-08.png)

4. 'Add model'을 클릭해서 앞 단계에서 S3로 업로드한 모델을 추가합니다.

![](./images/devday-deeplens-lab-09.png)

5. 모델 목록에서, 앞 단계에서 정의한 모델 이름을 선택하고 'Add model' 버튼을 클릭합니다.

![](./images/devday-deeplens-lab-14.png)

6. 프로젝트를 DeepLens에 배포하기 위해서, 프로젝트 목록에서 Object-detection을 선택한 후, 'Deploy to Device'를 클릭합니다.

![](./images/devday-deeplens-lab-15.png)

7. 'Target device' 목록에서 본인 그룹의 DeepLens 디바이스 이름을 선택하고, 'Review'를 클릭합니다. 그리고 그 다음 화면에서 'Deploy'를 클릭합니다.

![](./images/devday-deeplens-lab-16.png)

8. 프로젝트 배포 상황은 웹브라우저 상단의 파란색 상태창에 출력되는데, 배포가 완료되면 상태창이 녹색으로 바뀝니다.

![](./images/devday-deeplens-lab-17.png)
![](./images/devday-deeplens-lab-18.png)

9. 수행 결과 확인

DeepLens에 연결된 모니터을 통해서 결과를 확인하기 위해서는 DeepLens에 로그인해서 아래의 명령을 수행합니다.

```
mplayer -demuxer lavf -lavfdopts format=mjpeg:probesize=32 /tmp/results.mjpeg
```

![](./images/devday-deeplens-lab-20.png)

모델이 성공적으로 deploy 되었을 경우, 실시간으로 SSD object detection 알고리즘이 동작하는 화면을 모니터에서 확인할 수 있습니다.

이로써 오늘의 모든 실습을 마칩니다. 오랜 시간 수고 많으셨습니다.

## 참고

* 실습중 DeepLens 장비의 연결 상태에 문제가 생기면 아래 명령어로 Greengrass 서비스를 재시작하실 수 있습니다.

```
sudo systemctl restart greengrassd.service --no-block
```